# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [2]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `C:\Users\bonet\.julia\registries\General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving

 package versions...
  No Changes to `C:\Users\bonet\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\bonet\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...


  No Changes to `C:\Users\bonet\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\bonet\.julia\environments\v1.8\Manifest.toml`


### Récupération des données

In [3]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [14]:
function TestsSondabilite_relaxlin(model, Bestsol, BestProfit)
    TA, TO, TR = false, false, false
    if(!model.feasable)#Test de faisabilite
        TA=true
        println("TA")
    elseif(model.bound <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    #elseif( prod(abs.([round.(v, digits=0) for v in listvals]-listvals) .<= fill(10^-5, size(listvals))) 
    elseif(checksol(model))
        #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objectif(model) >= BestProfit)
            Bestsol = value.(model.sortedobjects)
            #BestProfit=value(benef)
            BestProfit=objectif(model)
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        #println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

TestsSondabilite_relaxlin (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [5]:

function SeparerNoeud_relaxlin(model, listobjs, listvals)
    # this obj is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-obj at the left  
    
    local nextObj
    for (i, obj) in enumerate(model.objects)
        if (!obj.fixed) && (model.borne1 || (abs(round(obj.value, digits=0) - obj.value) > 10^-5))
            setObj(model,i,1.0)
            nextObj = obj
            break
        end
    end

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,nextObj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
    listobjs, listvals
end


function ExplorerAutreNoeud_relaxlin(model, listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        val=pop!(listvals)
        node=pop!(listnodes)
        freeObj(model, obj, 0.0, 1.0)

        #go to right child if possible, otherwise go back to parent
        while( (val==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            val=pop!(listvals)
            node=pop!(listnodes)
            freeObj(model, obj, 0.0, 1.0)
        end
        if val==1.0
            setObj(model, obj, 0.0)
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return listobjs, listvals, listnodes, stop
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [6]:
mutable struct Object
    value :: Float64
    minValue :: Float64
    maxValue :: Float64
    index :: Int64
    fixed :: Bool
end
function value(obj)
    return obj.value
end

function name(obj)
    return string("x(", obj.index, ")")
end

mutable struct newModel
    objects :: Array{Object,1}
    sortedobjects :: Array{Object,1}
    bound ::Float64
    weight :: Array{Int64,1}
    price :: Array{Int64,1}
    capacity :: Int64
    borne1 :: Bool
    nb_rest :: Int64
    feasable :: Bool
end

function objectif(model)
    return sum([obj.value*model.price[obj.index] for obj in model.sortedobjects])
end

function setObj(model, obj::Object, value)
    obj.fixed = true
    obj.value = value
    obj.maxValue = value
    obj.minValue = value
    model.nb_rest -= 1
end

function setObj(model, index::Int64, value)
    obj = model.sortedobjects[index]
    setObj(model, obj, value)
end

function freeObj(model, obj, min, max)
    obj.fixed = false
    obj.minValue = min
    obj.maxValue = max
    model.nb_rest += 1
end

function CreationModele_relaxlin(price, weight, capacity, borne1)
    tri_par_ratio = sortperm(price./weight, rev=true)
    objs = []
    for i in 1:length(price)
        push!(objs, Object(0.0, 0.0, 1.0, i, false))
    end
    model = newModel(objs, view(objs, tri_par_ratio), -Inf, weight, price, capacity, borne1, length(price), false)
    return model
end

function checksol(model)
    prob_facile = false
    Contrainte_integer = false
    if model.borne1
        prob_facile = (model.nb_rest == 1)
        if prob_facile
            for (index, obj) in enumerate(model.sortedobjects)
                if !obj.fixed
                    listeObjet = []
                    listeValeur = []
                    for i in 1:length(model.price)
                        if model.objects[i].fixed
                            listeObjet = [listeObjet ; i]
                            listeValeur = [listeValeur ; model.objects[i].value]
                        end
                    end
                    # calcul de la valeur courante et de la capacité restante
                    valeur_courante = 0
                    capacite_restante = model.capacity
                    for i in 1:length(listeObjet)
                        if listeValeur[i] == 1
                            capacite_restante = capacite_restante - model.weight[listeObjet[i]]
                            valeur_courante = valeur_courante + model.price[listeObjet[i]]
                        end
                    end
                    poids = model.weight[obj.index]
                    obj.value = capacite_restante >= poids ? 1.0 : 0.0
                    break
                end
            end
        end
    else
        Contrainte_integer = (prod(abs.([round.(obj,digits=0) for obj in value.(model.sortedobjects)] - value.(model.sortedobjects)) .<= fill(10^-5, size(model.sortedobjects))))
    end
    return prob_facile || Contrainte_integer
end

# borne1 est un boolean mis à true si la borne supérieure à calculer est la borne 1 et false si c'est la borne 2
function borne_sup(model)
    listeObjet = []
    listeValeur = []
    for i in 1:length(model.price)
        if model.objects[i].fixed
            listeObjet = [listeObjet ; i]
            listeValeur = [listeValeur ; model.objects[i].value]
        end
    end
    # calcul de la valeur courante et de la capacité restante
    valeur_courante = 0
    capacite_restante = model.capacity
    for i in 1:length(listeObjet)
        if listeValeur[i] == 1
            capacite_restante = capacite_restante - model.weight[listeObjet[i]]
            valeur_courante = valeur_courante + model.price[listeObjet[i]]
        end
    end
    feasable = capacite_restante >= 10^-5
    if feasable
        # calcul de la borne sup 1
        if model.borne1
            for obj in model.sortedobjects
                if !(obj.fixed)
                    model.bound = valeur_courante + capacite_restante*(model.price[obj.index] / model.weight[obj.index])
                end
            end
        else
            # calcul de la borne sup 2
            for (index, obj) in enumerate(model.sortedobjects)
                if !(obj.fixed)
                    if capacite_restante <= 10^-5
                        obj.value = 0.0
                    else
                        poids = model.weight[obj.index]
                        if poids <= capacite_restante
                            obj.value = 1.0
                            capacite_restante -= poids
                        else
                            obj.value = capacite_restante / poids
                            capacite_restante = 0.0
                        end
                    end
                end
            model.bound = objectif(model)
            end
        end
    end 
model.feasable = feasable
end


borne_sup (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [12]:

function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    model = CreationModele_relaxlin(price, weight, capacity, true)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number=0
    stop = false

    while(!stop && current_node_number < 50)
    #while(!stop)

        #println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        #calcul de bornesup
        borne_sup(model)
        
        #println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")

        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_relaxlin(model, Bestsol, BestProfit)

        is_node_sondable = TA || TO || TR

        #println("\nIs node ", current_node_number, " sondable ? ", is_node_sondable, "\n")


        if(!is_node_sondable)
            listobjs, listvals = SeparerNoeud_relaxlin(model,listobjs,listvals)
        else
            listobjs, listvals, listnodes, stop = ExplorerAutreNoeud_relaxlin(model,listobjs,listvals,listnodes)        
        end
        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [15]:

for file in cd(readdir, "instanceTest/")
    println(file, " : ")
    BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instanceTest/"*file)
    println("\n******\n\nOptimal value = ", BestProfit)
end

knapPI_16_20_1000_1_-2291.opb.txt : 
TR

New Solution memorized [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 514.0

TR

New Solution memorized [1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 740.0

TR

New Solution memorized [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1143.0

TR

New Solution memorized [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,

 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1143.0

TR

New Solution memorized [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1143.0

TR

New Solution memorized [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1143.0

TR

New Solution memorized [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1880.0

TR

New Solution memorized [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 1880.0

TO
TO
TO
TO
TO
TO
TO
TO
TO
TO
TO
TO
TO


TO
TO
TO

FINISHED

******

Optimal value = 1880.0

Optimal x=[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

******

Optimal value = 1880.0
knapPI_16_20_1000_3_-2189.opb.txt : 

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = -1
knapPI_16_20_1000_5_-3230.opb.txt : 

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = 

-1
knapPI_4_50_1000_1_-994.opb.txt : 
TA
TA

FINISHED

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = -1
knapPI_4_50_1000_5_-1278.opb.txt : 

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = -1
knapPI_5_50_1000_1_-2096.opb.txt : 
TR

New Solution memorized [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] with bestprofit 236.0


******

Optimal value = 236.0

Optimal x=

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

******

Optimal value = 236.0
knapPI_5_50_1000_4_-1872.opb.txt : 

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = -1
knapPI_6_50_1000_1_-994.opb.txt : 
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA
TA

******

Optimal value = -1

Optimal x=Any[]

******

Optimal value = -1


## Questions préliminaires

### Question 1
Dans le Branch-and-Bound fourni initialement dans le notebook, la règle de séparation consiste à prendre les objets dans l'ordre.

### Question 2
La méthode de calcul de la borne supérieure est basée sur la solution du problème linéaire sans contraintes d'intégrité et dans lequel on autorise la fractionnalité des objets.

### Question 3
TA : Test d'admissibilité réussi si la capacité restante est strictement négative.
TO : Test d'optimalité réussi la borne supérieure est pire que la meilleure solution connue
TR : Test de résolution réussi quand toutes les valeurs des variables sont entières (à 10^-5 près)

### Question 4
La stratégie d'exploration correspond à du depth-first exploration research : recherche en profondeur avec priorité au noeud de gauche.

## Code et analyse 

### Question 3
-Pour la règle de séparation, on choisit les variables dans l'ordre décroissant du ratio valeur/poids. 

-Pour la stratégie d'exploration, on effectue un parcours en profondeur, en explorant d'abord les noeuds pour lesquels on décide de mettre l'objet (celui qui a servi de critère de séparation) dans le sac.

-Pour le test d'admissibilité, on vérifie si le problème n'est pas réalisable (capacité restante < 0)

-Pour le test d'optimalité, on vérifie si la borne supérieure est inférieure à la meilleure solution connue

-Pour le test de résolution, on vérifie si on est arrivé sur une feuille de l'arbre, c'est à dire qu'il ne reste plus qu'un seul objet à prendre (ou non si la capacité restante ne le permet pas).

### Question 4
On a décidé de garder un modèle, qui est modélisé par une structure de données mutable, car les valeurs de ses données changent au cours de l'exécution du programme. Les données du modèle sont notamment : une liste d'objets, on retrouve aussi la liste des poids, des prix, la capacité, la borne supérieure ou encore le nombre d'objets restant qui n'a pas encore été mis dans le sac et un booléen qui indique si le modèle peut se résoudre ou non. Objet est lui-même une structure de données mutable qui contient le a valeur de l'objet, son indice et un booléen pour savoir s'il est utilisé ou non. Les tests de sondabilité restent des booléens.

### Question 5
Malheureusement, nous n'avons pas réussi à faire fonctionner la borne 2. En revanche, on s'attend à ce que la borne 2 rende l'algorithme plus efficace. En effet, la borne 2 correspond à la solution du problème linéaire avec contraintes d'intégrité, mais qui autorise comme la borne 1 la fractionnalité des objets. Ce qui veut dire que la borne 1 qui possède moins de contraintes sera normalement plus grande que la borne 2, ainsi, on s'attend à ce que le test d'optimalité soit plus souvent réussi avec la borne 2 que la borne 1 et donc que l'algorithme effectue moins de séparation de noeud quand il utilise la borne 2, ce qui lui permettra d'aller plus vite dans sa résolution et donc d'avoir de meilleures performances.
